In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
PATH_bva = "data/annotations/20180415_bva.json"
PATH_cyber = "data/annotations/20180415_cyber_crime.json"
PATH_intellectual = "data/annotations/20180415_intellectual_property.json"
PATH_scotus = "data/annotations/20180415_scotus.json"

train_PATHS = [PATH_bva, PATH_intellectual, PATH_scotus]
test_PATHS = [PATH_cyber]

In [7]:
window_size = 6

count_ = {".":[0,0], ":":[0,0], "!":[0,0], ";":[0,0]}
def dataset_gen(x, window_size=10):
    dataset = json.load(open(x, "r", encoding="utf8"))
    x = []
    y = []

    for doc in dataset:
        text = dataset[doc]['text'].replace("\r","\n").replace("\t","\n")
        ends = []

        for loc in dataset[doc]['annotations']:
            ends.append(loc['end'])

        for pos, char in enumerate(text):
            if char in count_.keys():
                count_[char][0]+=1

                right = text[pos+1:pos+window_size+1]
                left = text[pos-window_size-1:pos-1]

                if len(right) == 0:
                    right = "\n"+ " "*(window_size-1)

                # elif right.replace(".","")[0] == "\n":
                #     continue
                # elif right[0] == "\n":
                #     right = "\n"+ " "*(window_size-1)

                right = right + " "*(window_size-len(right))
                left = " "*(window_size-len(left)) + left

                x.append([left, right])

                if pos+1 in ends:
                    y.append(1) # setence eneded if 1
                    count_[char][1]+=1
    
                else:
                    y.append(0) 

    return x, y

test_x,test_y = dataset_gen(test_PATHS[0], window_size=window_size)

train_x, train_y = [], []

for paths in train_PATHS:
    x,y = dataset_gen(paths, window_size=window_size)
    train_x = train_x + x
    train_y = train_y + y

print(len(train_x), len(test_x))

36341 12409


In [8]:
count_

{'.': [45048, 17835], ':': [1221, 287], '!': [28, 5], ';': [2453, 18]}

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

window_size = 6

def dataset_gen(x, window_size=10):
    dataset = json.load(open(x, "r", encoding="utf8"))
    x = []
    y = []

    for doc in dataset:
        text = dataset[doc]['text'].replace("\r","\n").replace("\t","\n")
        ends = []

        for loc in dataset[doc]['annotations']:
            ends.append(loc['end'])

        for pos, char in enumerate(text):
            if char == ".":
                right = text[pos+1:pos+window_size+1]
                left = text[pos-window_size-1:pos-1]

                if len(right) == 0:
                    right = "\n"+ " "*(window_size-1)

                # elif right.replace(".","")[0] == "\n":
                #     continue
                # elif right[0] == "\n":
                #     right = "\n"+ " "*(window_size-1)

                right = right + " "*(window_size-len(right))
                left = " "*(window_size-len(left)) + left

                x.append([left, right])

                if pos+1 in ends:
                    y.append(1) # setence eneded if 1
    
                else:
                    y.append(0) 

    return x, y

test_x,test_y = dataset_gen(test_PATHS[0], window_size=window_size)

train_x, train_y = [], []

for paths in train_PATHS:
    x,y = dataset_gen(paths, window_size=window_size)
    train_x = train_x + x
    train_y = train_y + y

print(len(train_x), len(test_x))



dataset = json.load(open(test_PATHS[0], "r", encoding="utf8"))


import string

characters = list(' '+string.printable.replace(" ",""))
characters.append('UNK')

characters = {x:i for i, x in enumerate(characters)}

print(len(characters))

def raw_str(string):
    return string.replace("\n","\\n").replace("\r","\\r").replace("\t","\\t")


class SBD_dataset(Dataset):
    def __init__(self, x, y, char_dict):
        self.x = x
        self.y = y
        self.len = len(x)
        self.char_dict = char_dict

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        x_ = ''.join(self.x[idx][0]) + ''.join(self.x[idx][1])
        
        x = []

        for i in x_:
            if i in self.char_dict:
                x.append(self.char_dict[i])

            else:
                x.append(self.char_dict['UNK'])

        # if len(x)!=20:
        #     print(self.x[idx])
        #     print(len(x))
        return torch.IntTensor(x), self.y[idx]


batch_size = 32

train_dataset = SBD_dataset(train_x[:-3000], torch.FloatTensor(train_y[:-3000]), characters)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

validation_dataset = SBD_dataset(train_x[-3000:], torch.FloatTensor(train_y[-3000:]), characters)
vailidation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

test_dataset = SBD_dataset(test_x, torch.FloatTensor(test_y), characters)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


33389 11659
101


In [13]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self,hidden_size,n_layer,vocab_size,embedding_size, kernel_size = 5):
        super().__init__()
        self.n_layer = n_layer
        self.hidden_size = hidden_size

        self.embedding_layer = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.cnn_layer = nn.Conv1d(embedding_size, 6, kernel_size)
        self.l1 = nn.Linear((2*window_size-kernel_size+1)*6, hidden_size)
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(hidden_size,1)

    def forward(self,x):
        x = self.embedding_layer(x)
        x = x.permute(0, 2, 1)
        x = self.cnn_layer(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.l1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.out(x))

        return x

# model(torch.IntTensor(list(range(len(characters)))).reshape(1,-1))

lr = 0.001
epoch = 15

train_n_minibatches = train_dataloader.__len__()
validation_n_minibatches = vailidation_dataloader.__len__()
test_n_minibatches = test_dataloader.__len__()

device = "cuda"
model_name = "cnn"
model = CNN(250,1,len(characters), 128).to(device)

model.load_state_dict(torch.load("NeuralNetworks/saved_models/cnn", map_location='cuda'))

<All keys matched successfully>

In [14]:
import string
import numpy as np
import time

characters = list(' '+string.printable.replace(" ",""))
characters.append('UNK')

characters = {x:i for i, x in enumerate(characters)}


def BIL_model(model, text):
    text = text.replace("\r","\n").replace("\t","\n")

    x = []
    offsets = []
    
    for pos, char in enumerate(text):
        if char == ".":
            right = text[pos+1:pos+window_size+1]
            left = text[pos-window_size-1:pos-1]
            if len(right) == 0:
                right = "\n"+ " "*(window_size-1)
            # elif right.replace(".","")[0] == "\n":
            #     continue
            # elif right[0] == "\n":
            #     right = "\n"+ " "*(window_size-1)
            right = right + " "*(window_size-len(right))
            left = " "*(window_size-len(left)) + left

            x.append([left, right])
            offsets.append(pos)


    inputs = []
    for i in x:
        temp = []
        x_ = ''.join(i[0]) + ''.join(i[1])
        for i in x_:
            if i in characters:
                temp.append(characters[i])

            else:
                temp.append(characters['UNK'])
            
        inputs.append(np.asarray(temp))

    inputs = torch.IntTensor(np.asarray(inputs)).to(device)

    pred = model(inputs)

    pred = pred.cpu().detach().numpy().tolist()
    pred_offsets = []

    for each_pred in pred:
        pred_offsets.append(round(each_pred[0]))

    output = []
    start = 0

    for i,j in zip(offsets, pred_offsets):
        if j == 1:
            output.append(text[start:i+1])
            start = i+1

    return output


def BIL_TF_PN(original, pred):
    original = original.replace("\n"," ").split()
    pred = pred.split()

    # print(original)
    # print(pred)
    
    beggining = [0, 0, 0, 0] #TP, TN, FP, FN
    end = [0, 0, 0, 0]
    i_max = [0, 0, 0, 0]

    if original[0] == pred[0]:
        beggining[0]+=1
        i_max[1]+=1
        end[1]+=1

    else:
        beggining[2]+=1
        i_max[3]+=1

    if original[-1] == pred[-1]:
        end[0]+=1
        i_max[1]+=1
        beggining[1]+=1

    else:
        end[2]+=1
        i_max[3]+=1

    change = False
    for pos in range(len(original)):
        # print(i_max[0], len(pred))
        if i_max[0] == len(pred):
            beggining[3] =0
            break
        
        if original[pos] != pred[0]:
            continue

        else:
            change = True
        
        i = [0, 0, 0, 0] #TP, TN, FP, FN
        temp_beg = beggining
        temp_end = end

        tip = len(original) if original < pred else pred

        for nth, (x, y) in enumerate(zip(range(pos, len(original)), range(len(pred)))):
            if original[x]==pred[y]:
                i[0]+=1
                temp_beg[1]+=1
                temp_end[1]+=1

            else:
                i[2]+=1
                temp_beg[1]+=1
                if nth!=tip:
                    temp_end[3]+=1


        if i[0]>i_max[0]:
            i_max = i
            if temp_beg[3]==0 and pos!=0:
                temp_beg[3]+=1

        elif i[0]==i_max[0] and (i[2] == 0) :
            i_max = i


            beggining = temp_beg
            end = temp_end
        

    if not change:
        i_max[2] = len(pred)-1

    return beggining, i_max, len(original), len(pred),  end


def raw_str(string):
    return string.replace("\n","\\n").replace("\r","\\r").replace("\t","\\t")

def check_range(pointer_start, pointer_end, ranges):
    intersection_dict = {}
    for i in ranges:
        if pointer_start>=i['start'] and pointer_end>=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [i['end'] - pointer_start, abs(i['start']-pointer_start)]

        if pointer_start<=i['start'] and pointer_end>=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [i['end'] - i['start'], abs(i['start']-pointer_start)]
        
        if pointer_start>=i['start'] and pointer_end<=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [pointer_end - pointer_start, abs(i['start']-pointer_start)]

        if pointer_start<=i['start'] and pointer_end<=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [pointer_end - i['start'], abs(i['start']-pointer_start)]

    max_intersection_key = None
    max_intersection_len = -1
    max_intersection_start_dist = 1000000
    
    for i in intersection_dict.keys():
        if intersection_dict[i][0]>max_intersection_len or (intersection_dict[i][0]==max_intersection_len and max_intersection_start_dist<intersection_dict[i][1]):
            max_intersection_len =  intersection_dict[i][0]
            max_intersection_key = i
            max_intersection_start_dist = intersection_dict[i][1]

    return max_intersection_key


def BIL_accuracy(x):
    if sum(x) == 0:
        return 0
    return (x[0]+x[1])/sum(x)

def BIL_recall(x):  #TP, TN, FP, FN
    if x[0] == 0:
        return 0

    return x[0]/(x[0]+x[3])

def BIL_precision(x):
    if x[0] == 0:
        return 0

    return x[0]/(x[0]+x[2])

def BIL_f1_score(x):
    if x[0] == 0:
        return 0

    return x[0]/(x[0] + 0.5*(x[2]+x[3]))

def BIL_specificity(x):
    if x[1] == 0:
        return 0

    return x[1]/(x[1] + x[2])

def BIL_acc_recall_precision_f1(x):
    acc, rec, prec, f1, specificity_  = 0, 0, 0, 0, 0
    t = 0
    for i in x:
        acc = acc + BIL_accuracy(i)
        rec = rec + BIL_recall(i)
        prec = prec + BIL_precision(i)
        f1 = f1 + BIL_f1_score(i)
        specificity_ = specificity_ + BIL_specificity(i)
        

    return acc/len(x), rec/len(x), prec/len(x), f1/len(x), specificity_/len(x)


def BIL_results(model, timer=False):
    results = {}
    total_time = 0

    for path_ in test_PATHS:
        dataset = json.load(open(path_, "r", encoding="utf8"))
        results[path_] = []
        for doc in dataset:
            x = dataset[doc]['text'].replace("\r","\n").replace("\t","\n")
            y = []
            for sentence in dataset[doc]['annotations']:
                y.append(dataset[doc]['text'][sentence['start']:sentence['end']])

            start_time = time.time()
            pred = BIL_model(model, x)

            if timer:
                print("--- Model took %s seconds to predict ---" % (time.time() - start_time))
            
            total_time = total_time + (time.time() - start_time)
            # print(len(y), len(pred))


            pointer = 0
            mappings = []
            filtred_pred = []

            for pred_sentences in pred:
                maped = check_range(pointer, pointer+len(pred_sentences), dataset[doc]['annotations'])
                # if maped != None:
                #     print(raw_str(pred_sentences), raw_str(dataset['59bd4ac35116540935ee6851']['text'][int(maped.split('-')[0]):int(maped.split('-')[1])]), sep=" ====== ", end="\n\n")

                pointer+=len(pred_sentences)

                pred_sentences = pred_sentences.replace("\n"," ")

                if maped is not None :
                    mappings.append(dataset[doc]['text'][int(maped.split('-')[0]):int(maped.split('-')[1])])
                    filtred_pred.append(pred_sentences)
                    results[path_].append(BIL_TF_PN(mappings[-1], filtred_pred[-1]))

                    print(filtred_pred[-1])
                    print(mappings[-1])
                    print("\n\n")

    for paths in results:
        B = []
        I = []
        L = []

        for i in results[paths]:
            B.append(i[0])
            I.append(i[1])
            L.append(i[-1])


        print(f"B : {BIL_acc_recall_precision_f1(B)}")
        print(f"I : {BIL_acc_recall_precision_f1(I)}")
        print(f"L : {BIL_acc_recall_precision_f1(L)}\n")

        if timer:
             print(f"--- Model took a total of {total_time} seconds to predict all the documents ---")

        return BIL_acc_recall_precision_f1(B), BIL_acc_recall_precision_f1(I), BIL_acc_recall_precision_f1(L)


BIL_results(model, True)

--- Model took 1.5537831783294678 seconds to predict ---
United States v. Barsumyan, 517 F.3d 1154 (9th Cir. 2008) Court of Appeals for the Ninth Circuit Filed: February 28th, 2008 Precedential Status: Precedential Citations: 517 F.3d 1154 Docket Number: 07-50251 Author: Joseph Jerome Farris  517 F.3d 1154 (2008) UNITED STATES of America, Plaintiff-Appellee, v. Aram BARSUMYAN, aka Seal A(1), Defendant-Appellant.
UNITED STATES of America, Plaintiff-Appellee,
v.
Aram BARSUMYAN, aka Seal A(1), Defendant-Appellant.



  No. 07-50251.
No. 07-50251.



 United States Court of Appeals, Ninth Circuit.
United States Court of Appeals, Ninth Circuit.



  Argued and Submitted January 10, 2008.
Argued and Submitted January 10, 2008.



 Filed February 28, 2008.
Filed February 28, 2008.



 *1155 Davina T. Chen, Deputy Federal Public Defender, Los Angeles, CA, for the defendant-appellant.
Davina T. Chen, Deputy Federal Public Defender, Los Angeles, CA, for the defendant-appellant.



  Alexander A.

((0.9053223723647468,
  0.8215590742996346,
  0.8215590742996346,
  0.8215590742996346,
  0.9056827731146623),
 (0.8406808030024573,
  0.8400324807145757,
  0.8346723129937844,
  0.8352102989636667,
  0.006649026555405399),
 (0.9640736110786597,
  0.8691373819701909,
  0.8735282176207877,
  0.8694300990510494,
  0.8341963068954054))